In [1]:
import torch as T
import albumentations as A
import cv2
import numpy as np
import os
import random
from tqdm import tqdm


In [2]:
if T.cuda.is_available():
    device=T.device("cuda")
else:
    device=T.device("cpu")

print(device)

cuda


In [3]:
if T.cuda.is_available():
    device = T.device("cuda")
else:
    device = T.device("cpu")

print(f"Using device: {device}")

# ---------------------- Augmentation Pipelines -----------------------
env_aug = A.Compose([
    A.OneOf([
        A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), src_radius=200, p=0.5),
        A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.5),
        A.RandomFog(fog_coef_range=(0.2, 0.4), p=0.5),
    ], p=1),
    A.Resize(384,512)
], additional_targets={'mask': 'mask'})

geo_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.D4(p=1),
    A.ElasticTransform(alpha=300, sigma=10, interpolation=cv2.INTER_NEAREST, mask_interpolation=cv2.INTER_NEAREST,
                       same_dxdy=True, border_mode=cv2.BORDER_CONSTANT, fill=0, fill_mask=0, p=0.5),
    A.ISONoise(color_shift=[0.01, 0.05], intensity=[0.1, 0.5], p=0.5),
    A.Resize(384, 512)
], additional_targets={'mask': 'mask'})

# ---------------------- Folders -----------------------
folder_prefix = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384"
folders = ["train", "test", "validation"]
new_folders = ["train_new", "test_new", "validation_new"]
mask_folders = ["trainlabel", "testlabel", "validationlabel"]
new_mask_folders = ["trainlabel_new", "testlabel_new", "validationlabel_new"]
img_ext = ".jpg"
mask_ext = ".png"

# ---------------------- Helper Functions -----------------------
def get_image_files(folder, extension):
    return [os.path.join(folder, f) for f in os.listdir(folder)
            if os.path.splitext(f)[1].lower() == extension]

def load_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load {image_path}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def load_mask(mask_path):
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        raise ValueError(f"Failed to load {mask_path}")
    return mask

def save_image(path, image_rgb):
    image_bgr = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, image_bgr)

def save_mask(path, mask):
    cv2.imwrite(path, mask)

# ---------------------- Main Loop -----------------------
for folder_div, new_folder_div, mask_folder_div, new_mask_folder_div in zip(folders, new_folders, mask_folders, new_mask_folders):
    folder = os.path.join(folder_prefix, folder_div)
    new_folder = os.path.join(folder_prefix, new_folder_div)
    mask_folder = os.path.join(folder_prefix, mask_folder_div)
    new_mask_folder = os.path.join(folder_prefix, new_mask_folder_div)

    if not os.path.isdir(folder) or not os.path.isdir(mask_folder):
        print(f"Skipping invalid folder: {folder} or {mask_folder}")
        continue

    os.makedirs(new_folder, exist_ok=True)
    os.makedirs(new_mask_folder, exist_ok=True)

    print(f"\n📁 Processing folder: {folder_div}")
    image_paths = get_image_files(folder, img_ext)
    image_paths.sort()

    images, masks, names = [], [], []

    # Load original data
    for path in image_paths:
        image = load_image(path)
        basename = os.path.splitext(os.path.basename(path))[0]
        mask_path = os.path.join(mask_folder, basename + mask_ext)
        mask = load_mask(mask_path)

        images.append(image)
        masks.append(mask)
        names.append(basename)

    orig_count = len(images)
    all_aug_images, all_aug_masks, all_aug_names = [], [], []

    # ----------------------- Environmental Augmentations -----------------------
    for i, (image, mask, basename) in enumerate(tqdm(list(zip(images, masks, names)), desc="🌦️ Environmental Aug")):
        augmented = env_aug(image=image, mask=mask)
        all_aug_images.append(augmented['image'])
        all_aug_masks.append(augmented['mask'])
        all_aug_names.append(f"{orig_count + i + 1}_env")

    # ----------------------- Geometric Augmentations -----------------------
    current_max_id = orig_count * 2
    for i, (img, msk, _) in enumerate(tqdm(list(zip(images + all_aug_images, masks + all_aug_masks, names + all_aug_names)), desc="🌀 Geometric Aug")):
        augmented = geo_aug(image=img, mask=msk)
        all_aug_images.append(augmented['image'])
        all_aug_masks.append(augmented['mask'])
        all_aug_names.append(f"{current_max_id + i + 1}_geo")

    combined = list(zip(images + all_aug_images, masks + all_aug_masks, names + all_aug_names))
    random.shuffle(combined)

    for i, (img, msk, _) in enumerate(tqdm(combined, desc="💾 Saving")):
        new_name = f"{i + 1}_image"
        save_image(os.path.join(new_folder, new_name + img_ext), img)
        save_mask(os.path.join(new_mask_folder, new_name + mask_ext), msk)

    print(f"✅ {folder_div}: Original = {orig_count} | Final Augmented = {len(combined)} (4x)")



Using device: cuda

📁 Processing folder: train


D:\Python\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
💾 Saving: 100%|███████████████████████████████████████████████████████████████████| 1600/1600 [00:20<00:00, 79.93it/s]


✅ train: Original = 400 | Final Augmented = 1600 (4x)

📁 Processing folder: test


💾 Saving: 100%|██████████████████████████████████████████████████████████████████| 1200/1200 [00:08<00:00, 135.42it/s]


✅ test: Original = 300 | Final Augmented = 1200 (4x)

📁 Processing folder: validation


💾 Saving: 100%|████████████████████████████████████████████████████████████████████| 352/352 [00:01<00:00, 206.57it/s]

✅ validation: Original = 88 | Final Augmented = 352 (4x)
